In [5]:
import pandas as pd
import numpy as np
import psycopg2
import math

#plotly
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'


mapbox_token = 'pk.eyJ1Ijoiam9ldGhlcXVhbnQiLCJhIjoiY2w2MWc5N3lyMDFhYjNicDVheXExOTVmbyJ9.uhlvKEoKoQ1j5BCe5wn_ZQ'

In [6]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)
cursor = connection.cursor()

In [7]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [8]:
rollback_before_flag = True
rollback_after_flag = True

query = """
select * from stations
"""

stations = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
stations = stations.set_index('station')
stations.head()

,latitude,longitude,transfer_time
station,,,
12th Street,37.803608,-122.272006,282
16th Street Mission,37.764847,-122.420042,287
19th Street,37.807869,-122.268980,67
24th Street Mission,37.752000,-122.418700,277
Antioch,37.996281,-121.783404,0


In [9]:
station_calculated_stats = pd.read_csv('../../processed_data/df_features.csv')
station_calculated_stats = station_calculated_stats.rename(columns={'node': 'station'})
station_calculated_stats = station_calculated_stats.set_index('station')
station_stats = pd.concat([stations, station_calculated_stats], axis=1)
station_stats.head()

,latitude,longitude,transfer_time,Unnamed: 0,degree,avg_degree,std_degree,closeness,avg_closeness,std_closeness,...,std_triangle_count,clustering_coefficient,avg_clustering_coefficient,std_clustering_coefficient,community,within_1m_population,within_2m_population,within_3m_population,within_4m_population,within_5m_population
station,,,,,,,,,,,,,,,,,,,,,
12th Street,37.803608,-122.272006,282,0,3,2.04,0.493219,0.2333,0.169662,0.033104,...,0.328261,0.3333,0.066664,0.212955,29,16062.0,175958.0,298398.0,367715.0,510498.0
16th Street Mission,37.764847,-122.420042,287,1,2,2.04,0.493219,0.1712,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,10,63489.0,339093.0,554106.0,826249.0,870044.0
19th Street,37.807869,-122.268980,67,2,2,2.04,0.493219,0.2195,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,46,16062.0,165215.0,298398.0,420109.0,576172.0
24th Street Mission,37.752000,-122.418700,277,3,2,2.04,0.493219,0.1663,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,10,108915.0,315201.0,621661.0,823185.0,989138.0
Antioch,37.996281,-121.783404,0,4,1,2.04,0.493219,0.1069,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,31,0.0,66933.0,110721.0,110721.0,152632.0


In [10]:
station_stats.sort_values('within_1m_population')

,latitude,longitude,transfer_time,Unnamed: 0,degree,avg_degree,std_degree,closeness,avg_closeness,std_closeness,...,std_triangle_count,clustering_coefficient,avg_clustering_coefficient,std_clustering_coefficient,community,within_1m_population,within_2m_population,within_3m_population,within_4m_population,within_5m_population
station,,,,,,,,,,,,,,,,,,,,,
Coliseum,37.753611,-122.196944,54,11,3,2.04,0.493219,0.2165,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,23,0.0,123040.0,267162.0,410020.0,495903.0
El Cerrito del Norte,37.925183,-122.316939,311,17,2,2.04,0.493219,0.1426,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,2,0.0,81472.0,140593.0,268588.0,369372.0
Milpitas,37.410278,-121.891111,292,28,2,2.04,0.493219,0.1323,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,45,0.0,31170.0,164682.0,164682.0,435506.0
OAK,37.713200,-122.212200,0,32,1,2.04,0.493219,0.1683,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,23,0.0,84499.0,154627.0,197739.0,410051.0
Daly City,37.706224,-122.468934,285,14,2,2.04,0.493219,0.1530,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,14,0.0,231056.0,330204.0,616010.0,805122.0
Concord,37.973745,-122.029127,0,13,2,2.04,0.493219,0.1558,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,31,0.0,87117.0,122213.0,188050.0,188050.0
Pittsburg,38.018869,-121.944208,0,34,2,2.04,0.493219,0.1400,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,31,0.0,0.0,96081.0,116202.0,187733.0
Castro Valley,37.690748,-122.075679,0,9,2,2.04,0.493219,0.1799,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,23,0.0,110328.0,165604.0,290193.0,403433.0
West Dublin,37.699726,-121.928273,0,48,2,2.04,0.493219,0.1584,0.169662,0.033104,...,0.328261,0.0000,0.066664,0.212955,23,0.0,54420.0,90736.0,171748.0,229219.0


In [11]:
station_stats.columns

Index(['latitude', 'longitude', 'transfer_time', 'Unnamed: 0', 'degree',
       'avg_degree', 'std_degree', 'closeness', 'avg_closeness',
       'std_closeness', 'betweenness', 'avg_betweenness', 'std_betweenness',
       'triangle_count', 'avg_triangle_count', 'std_triangle_count',
       'clustering_coefficient', 'avg_clustering_coefficient',
       'std_clustering_coefficient', 'community', 'within_1m_population',
       'within_2m_population', 'within_3m_population', 'within_4m_population',
       'within_5m_population'],
      dtype='object')

In [12]:
px.set_mapbox_access_token(mapbox_token)
fig = px.scatter_mapbox(station_stats.reset_index(), lat="latitude", lon="longitude",color="betweenness", size="within_1m_population", hover_name="station",
                  color_continuous_scale=px.colors.sequential.BuPu, size_max=30, zoom=10, width=1000, height=1000, )

fig.update_layout(
        title_text = 'Station Betweenness and Population within 1 Mile',
        showlegend = True,
    )
fig.show()

In [13]:
px.set_mapbox_access_token(mapbox_token)
fig = px.scatter_mapbox(station_stats.reset_index(), lat="latitude", lon="longitude",color="community", size="within_1m_population", hover_name="station",
                  color_continuous_scale=px.colors.sequential.BuPu, size_max=30, zoom=10, width=1000, height=1000, )

fig.update_layout(
        title_text = 'Station Betweenness and Population within 1 Mile',
        showlegend = True,
    )
fig.show()

In [17]:
zip_populations = pd.read_csv('../../processed_data/zip_num_stations.csv')
zip_populations.columns

Index(['Unnamed: 0', 'zip', 'latitude', 'longitude', 'city', 'state',
       'population', 'area', 'density', 'time_zone', 'within_1m_station',
       'within_2m_station', 'within_3m_station', 'within_4m_station',
       'within_5m_station'],
      dtype='object')

In [28]:
zip_populations_ca_not_stations = zip_populations[(zip_populations.within_2m_station == 0) & (zip_populations.state == 'CA')]

In [30]:
px.set_mapbox_access_token(mapbox_token)
fig = px.scatter_mapbox(zip_populations_ca_not_stations.reset_index(), lat="latitude", lon="longitude",color="population", size="population", hover_name="city",
                  color_continuous_scale=px.colors.sequential.BuPu, size_max=30, zoom=10, width=1000, height=1000, )

fig.update_layout(
        title_text = 'Population of Zip Codes, Station Greater Than 2 Miles Away',
        showlegend = True,
    )
fig.show()